In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Veri setini yükleyin
data = pd.read_csv('jobs_in_data.csv')


In [43]:
data.head()

,work_year,job_title,job_category,salary_currency,salary,salary_in_usd,employee_residence,experience_level,employment_type,work_setting,company_location,company_size
0,2023,Data DevOps Engineer,Data Engineering,EUR,88000,95012,Germany,Mid-level,Full-time,Hybrid,Germany,L
1,2023,Data Architect,Data Architecture and Modeling,USD,186000,186000,United States,Senior,Full-time,In-person,United States,M
2,2023,Data Architect,Data Architecture and Modeling,USD,81800,81800,United States,Senior,Full-time,In-person,United States,M
3,2023,Data Scientist,Data Science and Research,USD,212000,212000,United States,Senior,Full-time,In-person,United States,M
4,2023,Data Scientist,Data Science and Research,USD,93300,93300,United States,Senior,Full-time,In-person,United States,M


In [44]:
data = data.drop(['salary'], axis=1)

In [45]:
data = data.drop(['salary_currency','employee_residence'], axis=1)

In [47]:
data = data.drop(['work_year'], axis=1)

In [49]:
data = data.drop(['job_category'], axis=1)

In [50]:
data.head()

,job_title,salary_in_usd,experience_level,employment_type,work_setting,company_location,company_size
0,Data DevOps Engineer,95012,Mid-level,Full-time,Hybrid,Germany,L
1,Data Architect,186000,Senior,Full-time,In-person,United States,M
2,Data Architect,81800,Senior,Full-time,In-person,United States,M
3,Data Scientist,212000,Senior,Full-time,In-person,United States,M
4,Data Scientist,93300,Senior,Full-time,In-person,United States,M


In [58]:

# 'experience_level' ve 'company_size' sütunları için one-hot encoding uygulayın
data_with_dummies = pd.get_dummies(data, columns=['experience_level', 'company_size','employment_type','work_setting'])

# 'job_title' sütununu kategorik bir değişkenden sayısal bir değişkene dönüştürün
data_with_dummies['job_title'] = data_with_dummies['job_title'].astype('category').cat.codes
data_with_dummies['company_location'] = data_with_dummies['company_location'].astype('category').cat.codes


In [59]:
data_with_dummies.head()

,job_title,salary_in_usd,company_location,experience_level_Entry-level,experience_level_Executive,experience_level_Mid-level,experience_level_Senior,company_size_L,company_size_M,company_size_S,employment_type_Contract,employment_type_Freelance,employment_type_Full-time,employment_type_Part-time,work_setting_Hybrid,work_setting_In-person,work_setting_Remote
0,42,95012,23,False,False,True,False,True,False,False,False,False,True,False,True,False,False
1,41,186000,69,False,False,False,True,False,True,False,False,False,True,False,False,True,False
2,41,81800,69,False,False,False,True,False,True,False,False,False,True,False,False,True,False
3,69,212000,69,False,False,False,True,False,True,False,False,False,True,False,False,True,False
4,69,93300,69,False,False,False,True,False,True,False,False,False,True,False,False,True,False


In [60]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [61]:
# Hedef değişkeni (salary_in_usd) ve özellikleri ayırın
X = data_with_dummies.drop('salary_in_usd', axis=1)
y = data_with_dummies['salary_in_usd']

In [70]:
import numpy as np

In [110]:
# Eğitim ve test setlerini ayırın
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost modelini oluşturun
xgb_model = XGBRegressor(n_estimators=100,  # Ağaç sayısı (Düzenleyin)
                        max_depth=5,       # Ağaç derinliği (Düzenleyin)
                        learning_rate=0.1, # Öğrenme oranı (Düzenleyin)
                        #subsample=0.8,     # Rastgele örneklemleme oranı (Düzenleyin)
                        #colsample_bytree=0.8, # Sütun örnekleme oranı (Düzenleyin)
                        #gamma=0.0,         # Gamma (Düzenleyin)
                        #reg_alpha=0.1,     # L1 düzenleme (Düzenleyin)
                        #reg_lambda=0.1,    # L2 düzenleme (Düzenleyin)
                        #min_child_weight=1) # Minimum çocuk ağırlığı (Düzenleyin)
                        )

# Modeli eğitim verileriyle eğitin
xgb_model.fit(X_train, y_train)

# Test verileri üzerinde tahmin yapın
y_pred = xgb_model.predict(X_test)

# Tahmin performansını MSE kullanarak değerlendirin
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Örnek bir veri girdisi (verileri kendi verilerinizle değiştirin)
input_data = [41,23,False, False, True, False, True, False, False,False,False,True,False,True,False,False]

# Veriyi NumPy dizisine çevirin
input_data = np.array(input_data).reshape(1, -1)

# Tahmin yapın
predicted_salary = xgb_model.predict(input_data)

# Tahmini maaşı yazdırın
print(f'Predicted Salary: {predicted_salary[0]:.2f}')

Mean Squared Error: 2646371973.361902
Predicted Salary: 73945.15


In [111]:
# Örnek bir veri girdisi (verileri kendi verilerinizle değiştirin)
input_data = [41,23,False, False, True, False, True, False, False,False,False,True,False,True,False,False]

# Veriyi NumPy dizisine çevirin
input_data = np.array(input_data).reshape(1, -1)

# Tahmin yapın
predicted_salary = xgb_model.predict(input_data)

# Tahmini maaşı yazdırın
print(f'Predicted Salary: {predicted_salary[0]:.2f}')

Predicted Salary: 73945.15


In [112]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2024-01-16 20:10:53.230416: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-16 20:10:53.284234: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 20:10:53.901169: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [113]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Yapay Sinir Ağı modelini oluşturun
model = keras.Sequential([
    layers.Input(shape=X_train.shape[1]),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Çıkış katmanı, çünkü regresyon yapılıyor
])

# Modeli derleyin
model.compile(optimizer='adam', loss='mean_squared_error')

# Modeli eğitin
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=2)

# Modeli test verileri üzerinde değerlendirin
y_pred = model.predict(X_test)

# Tahmin performansını MSE kullanarak değerlendirin
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Epoch 1/100


2024-01-16 20:11:10.026076: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-16 20:11:10.038186: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


188/188 - 1s - loss: 26365939712.0000 - val_loss: 26428956672.0000 - 928ms/epoch - 5ms/step
Epoch 2/100
188/188 - 0s - loss: 26238427136.0000 - val_loss: 26158092288.0000 - 350ms/epoch - 2ms/step
Epoch 3/100
188/188 - 0s - loss: 25747191808.0000 - val_loss: 25395834880.0000 - 384ms/epoch - 2ms/step
Epoch 4/100
188/188 - 0s - loss: 24677451776.0000 - val_loss: 23976232960.0000 - 370ms/epoch - 2ms/step
Epoch 5/100
188/188 - 0s - loss: 22928166912.0000 - val_loss: 21854210048.0000 - 390ms/epoch - 2ms/step
Epoch 6/100
188/188 - 0s - loss: 20526977024.0000 - val_loss: 19134025728.0000 - 371ms/epoch - 2ms/step
Epoch 7/100
188/188 - 0s - loss: 17645721600.0000 - val_loss: 16066223104.0000 - 390ms/epoch - 2ms/step
Epoch 8/100
188/188 - 0s - loss: 14564297728.0000 - val_loss: 12944340992.0000 - 370ms/epoch - 2ms/step
Epoch 9/100
188/188 - 0s - loss: 11599576064.0000 - val_loss: 10106078208.0000 - 356ms/epoch - 2ms/step
Epoch 10/100
188/188 - 0s - loss: 9019140096.0000 - val_loss: 7782708736.000

In [114]:
input_data = [41, 23, False, False, True, False, True, False, False, False, False, True, False, True, False, False]

# Veriyi NumPy dizisine çevirin
input_data = np.array(input_data).reshape(1, -1)

# Veriyi ölçeklendirin (verileri ölçeklendirdiğinizden emin olun)
input_data = scaler.transform(input_data)  # Daha önce kullanılan ölçekleyiciyi kullanın

# Tahmin yapın
predicted_salary = model.predict(input_data)

# Tahmini maaşı yazdırın
print(f'Predicted Salary: {predicted_salary[0][0]:.2f}')

1/1 [==============================] - 0s 24ms/step
Predicted Salary: 80897.50


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
